# K-mer 
Basic K-mer counting.

In [1]:
import time
def show_time():
    t = time.time()
    print(time.strftime('%Y-%m-%d %H:%M:%S %Z', time.localtime(t)))
show_time()

2021-07-07 11:39:43 EDT


In [2]:
PC_SEQUENCES=32000
NC_SEQUENCES=32000
RNA_LEN=32
CDS_LEN=16

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
import sys
IN_COLAB = False
try:
    from google.colab import drive
    IN_COLAB = True
except:
    pass
if IN_COLAB:
    print("On Google CoLab, mount cloud-local file, get our code from GitHub.")
    PATH='/content/drive/'
    #drive.mount(PATH,force_remount=True)  # hardly ever need this
    #drive.mount(PATH)    # Google will require login credentials
    DATAPATH=PATH+'My Drive/data/'  # must end in "/"
    import requests
    r = requests.get('https://raw.githubusercontent.com/ShepherdCode/Soars2021/master/SimTools/RNA_describe.py')
    with open('RNA_describe.py', 'w') as f:
        f.write(r.text)  
    from RNA_describe import Random_Base_Oracle
else:
        print("CoLab not working. On my PC, use relative paths.")
        DATAPATH='data/'  # must end in "/"
        sys.path.append("..") # append parent dir in order to use sibling dirs
        from SimTools.RNA_describe import Random_Base_Oracle
MODELPATH="BestModel"  # saved on cloud instance and lost after logout
#MODELPATH=DATAPATH+MODELPATH  # saved on Google Drive but requires login

CoLab not working. On my PC, use relative paths.


## K-mer counting

### Functions to create the dict of {kmer:count}

In [5]:
def make_kmer_keys(K):
    shorter_kmers=['']
    for i in range(K):
        longer_kmers=[]
        for mer in shorter_kmers:
            # No support for N or any non-ACGT bases.
            longer_kmers.append(mer+'A')
            longer_kmers.append(mer+'C')
            longer_kmers.append(mer+'G')
            longer_kmers.append(mer+'T')
        shorter_kmers = longer_kmers
    return shorter_kmers
def make_kmer_dict(keys,init=0):
    return dict.fromkeys(keys,init)
def make_dict_upto_K(max_K):
    keys=make_kmer_keys(1)
    for k in range(2,max_K+1):
        keys.extend(make_kmer_keys(k))
    counts = make_kmer_dict(keys)
    return counts


### Naive K-mer counting algorithm
Algorithm:  
1. for every string  
    1. for every K  
        1. for every position  
            1. kmer=substring
            2. count{kmer}++

In [6]:
def update_count_one_K(counts,K,rna,tail=False):
    L = len(rna)
    padding=" "*(K-1)
    padded=rna+padding
    for i in range(0,L-K+1):
        kmer=padded[i:i+K]
        counts[kmer] += 1
    if tail and K>1:  
        # for Harvester algorithm, count last letters as special case
        for start_pos in range(L-K+1,L):
            for end_pos in range(start_pos+1,L+1):
                kmer=rna[start_pos:end_pos]
                counts[kmer] += 1
    return counts
def update_count_upto_K(counts,max_K,sample,tail=False):
    for i in range(1,max_K+1):
        update_count_one_K(counts,i,sample,tail)
    return counts

### Harvester K-mer counting algorithm
Algorithm:  
1. Count K-mers for max K only  
2. For each K-mer in counts table:  
    1. For every prefix of the K-mer:  
        1. count{prefix} += count{kmer}  
3. Handle last K-1 letters of each string as special case

In [7]:
def harvest_counts_from_K(counts,max_K):
    for kmer in counts.keys():
        klen = len(kmer)
        kcnt = counts[kmer]
        if klen==max_K and kcnt>0:
            for i in range(1,klen):
                prefix = kmer[:i]
                counts[prefix] += kcnt
    return counts

In [8]:
def count_to_frequency(counts,max_K):
    freqs = dict.fromkeys(counts.keys(),0.0)
    for k in range(1,max_K+1):
        tot = 0
        for kmer in counts.keys():
            if len(kmer)==k:
                tot += counts[kmer]
        for kmer in counts.keys():
            if len(kmer)==k:
                freqs[kmer] = 1.0*counts[kmer]/tot
    return freqs

## Demo

### Demo: Naive algorithm

In [9]:
MAX_K = 3
counts1 = make_dict_upto_K(MAX_K)
print("Initial counts:\n",counts1)

sample = "ACCGGGTTTTACGTACGT"
update_count_upto_K(counts1,MAX_K,sample)
print("Final counts:\n",counts1)

Initial counts:
 {'A': 0, 'C': 0, 'G': 0, 'T': 0, 'AA': 0, 'AC': 0, 'AG': 0, 'AT': 0, 'CA': 0, 'CC': 0, 'CG': 0, 'CT': 0, 'GA': 0, 'GC': 0, 'GG': 0, 'GT': 0, 'TA': 0, 'TC': 0, 'TG': 0, 'TT': 0, 'AAA': 0, 'AAC': 0, 'AAG': 0, 'AAT': 0, 'ACA': 0, 'ACC': 0, 'ACG': 0, 'ACT': 0, 'AGA': 0, 'AGC': 0, 'AGG': 0, 'AGT': 0, 'ATA': 0, 'ATC': 0, 'ATG': 0, 'ATT': 0, 'CAA': 0, 'CAC': 0, 'CAG': 0, 'CAT': 0, 'CCA': 0, 'CCC': 0, 'CCG': 0, 'CCT': 0, 'CGA': 0, 'CGC': 0, 'CGG': 0, 'CGT': 0, 'CTA': 0, 'CTC': 0, 'CTG': 0, 'CTT': 0, 'GAA': 0, 'GAC': 0, 'GAG': 0, 'GAT': 0, 'GCA': 0, 'GCC': 0, 'GCG': 0, 'GCT': 0, 'GGA': 0, 'GGC': 0, 'GGG': 0, 'GGT': 0, 'GTA': 0, 'GTC': 0, 'GTG': 0, 'GTT': 0, 'TAA': 0, 'TAC': 0, 'TAG': 0, 'TAT': 0, 'TCA': 0, 'TCC': 0, 'TCG': 0, 'TCT': 0, 'TGA': 0, 'TGC': 0, 'TGG': 0, 'TGT': 0, 'TTA': 0, 'TTC': 0, 'TTG': 0, 'TTT': 0}
Final counts:
 {'A': 3, 'C': 4, 'G': 5, 'T': 6, 'AA': 0, 'AC': 3, 'AG': 0, 'AT': 0, 'CA': 0, 'CC': 1, 'CG': 3, 'CT': 0, 'GA': 0, 'GC': 0, 'GG': 2, 'GT': 3, 'TA': 2, '

### Demo: Harvester algorithm

In [10]:
MAX_K = 3
counts2 = make_dict_upto_K(MAX_K)
print("Initial counts:\n",counts2)

sample = "ACCGGGTTTTACGTACGT"
update_count_one_K(counts2,MAX_K,sample,True)
print("Partial counts (just max K and special case letters)\n:",counts2)
harvest_counts_from_K(counts2,MAX_K)
print("Final counts (includes smaller values of K):\n",counts2)


Initial counts:
 {'A': 0, 'C': 0, 'G': 0, 'T': 0, 'AA': 0, 'AC': 0, 'AG': 0, 'AT': 0, 'CA': 0, 'CC': 0, 'CG': 0, 'CT': 0, 'GA': 0, 'GC': 0, 'GG': 0, 'GT': 0, 'TA': 0, 'TC': 0, 'TG': 0, 'TT': 0, 'AAA': 0, 'AAC': 0, 'AAG': 0, 'AAT': 0, 'ACA': 0, 'ACC': 0, 'ACG': 0, 'ACT': 0, 'AGA': 0, 'AGC': 0, 'AGG': 0, 'AGT': 0, 'ATA': 0, 'ATC': 0, 'ATG': 0, 'ATT': 0, 'CAA': 0, 'CAC': 0, 'CAG': 0, 'CAT': 0, 'CCA': 0, 'CCC': 0, 'CCG': 0, 'CCT': 0, 'CGA': 0, 'CGC': 0, 'CGG': 0, 'CGT': 0, 'CTA': 0, 'CTC': 0, 'CTG': 0, 'CTT': 0, 'GAA': 0, 'GAC': 0, 'GAG': 0, 'GAT': 0, 'GCA': 0, 'GCC': 0, 'GCG': 0, 'GCT': 0, 'GGA': 0, 'GGC': 0, 'GGG': 0, 'GGT': 0, 'GTA': 0, 'GTC': 0, 'GTG': 0, 'GTT': 0, 'TAA': 0, 'TAC': 0, 'TAG': 0, 'TAT': 0, 'TCA': 0, 'TCC': 0, 'TCG': 0, 'TCT': 0, 'TGA': 0, 'TGC': 0, 'TGG': 0, 'TGT': 0, 'TTA': 0, 'TTC': 0, 'TTG': 0, 'TTT': 0}
Partial counts (just max K and special case letters)
: {'A': 0, 'C': 0, 'G': 1, 'T': 1, 'AA': 0, 'AC': 0, 'AG': 0, 'AT': 0, 'CA': 0, 'CC': 0, 'CG': 0, 'CT': 0, 'GA': 

In [11]:
if counts1==counts2:
    print("Success. Harvester output matches naive results!")
else:
    print("Fail. Harvester output differs from naive results!")

Success. Harvester output matches naive results!


In [12]:
freqs = count_to_frequency(counts2,MAX_K)
print ("Frequency:\n",freqs)

Frequency:
 {'A': 0.16666666666666666, 'C': 0.2222222222222222, 'G': 0.2777777777777778, 'T': 0.3333333333333333, 'AA': 0.0, 'AC': 0.17647058823529413, 'AG': 0.0, 'AT': 0.0, 'CA': 0.0, 'CC': 0.058823529411764705, 'CG': 0.17647058823529413, 'CT': 0.0, 'GA': 0.0, 'GC': 0.0, 'GG': 0.11764705882352941, 'GT': 0.17647058823529413, 'TA': 0.11764705882352941, 'TC': 0.0, 'TG': 0.0, 'TT': 0.17647058823529413, 'AAA': 0.0, 'AAC': 0.0, 'AAG': 0.0, 'AAT': 0.0, 'ACA': 0.0, 'ACC': 0.0625, 'ACG': 0.125, 'ACT': 0.0, 'AGA': 0.0, 'AGC': 0.0, 'AGG': 0.0, 'AGT': 0.0, 'ATA': 0.0, 'ATC': 0.0, 'ATG': 0.0, 'ATT': 0.0, 'CAA': 0.0, 'CAC': 0.0, 'CAG': 0.0, 'CAT': 0.0, 'CCA': 0.0, 'CCC': 0.0, 'CCG': 0.0625, 'CCT': 0.0, 'CGA': 0.0, 'CGC': 0.0, 'CGG': 0.0625, 'CGT': 0.125, 'CTA': 0.0, 'CTC': 0.0, 'CTG': 0.0, 'CTT': 0.0, 'GAA': 0.0, 'GAC': 0.0, 'GAG': 0.0, 'GAT': 0.0, 'GCA': 0.0, 'GCC': 0.0, 'GCG': 0.0, 'GCT': 0.0, 'GGA': 0.0, 'GGC': 0.0, 'GGG': 0.0625, 'GGT': 0.0625, 'GTA': 0.0625, 'GTC': 0.0, 'GTG': 0.0, 'GTT': 0.06

## Demo on large dataset

In [13]:
rbo=Random_Base_Oracle(RNA_LEN,True)
pc_all,nc_all = rbo.get_partitioned_sequences(CDS_LEN,10) # just testing
pc_all,nc_all = rbo.get_partitioned_sequences(CDS_LEN,PC_SEQUENCES)
print("Use",len(pc_all),"PC seqs")
print("Use",len(nc_all),"NC seqs")

It took 46 trials to reach 10 per class.
It took 139902 trials to reach 32000 per class.
Use 32000 PC seqs
Use 32000 NC seqs


In [14]:
MAX_K = 3
pc_counts = make_dict_upto_K(MAX_K)
for sample in pc_all:
    update_count_one_K(pc_counts,MAX_K,sample,True)
harvest_counts_from_K(pc_counts,MAX_K)
print("PC counts:\n",pc_counts)
pc_freqs = count_to_frequency(pc_counts,MAX_K)
print ("Frequency:\n",pc_freqs)

PC counts:
 {'A': 284878, 'C': 204217, 'G': 261748, 'T': 273157, 'AA': 69738, 'AC': 50372, 'AG': 61835, 'AT': 94299, 'CA': 53193, 'CC': 44877, 'CG': 45058, 'CT': 53451, 'GA': 76375, 'GC': 56789, 'GG': 56239, 'GT': 64257, 'TA': 75725, 'TC': 44727, 'TG': 91217, 'TT': 53848, 'AAA': 15197, 'AAC': 13662, 'AAG': 13771, 'AAT': 24093, 'ACA': 12562, 'ACC': 10801, 'ACG': 11033, 'ACT': 13408, 'AGA': 15512, 'AGC': 13578, 'AGG': 13820, 'AGT': 15850, 'ATA': 18567, 'ATC': 10940, 'ATG': 48985, 'ATT': 13296, 'CAA': 11631, 'CAC': 9914, 'CAG': 9897, 'CAT': 20290, 'CCA': 11582, 'CCC': 9968, 'CCG': 9906, 'CCT': 11925, 'CGA': 11625, 'CGC': 9901, 'CGG': 9831, 'CGT': 12159, 'CTA': 17463, 'CTC': 9779, 'CTG': 12606, 'CTT': 12065, 'GAA': 17206, 'GAC': 15355, 'GAG': 15352, 'GAT': 25846, 'GCA': 14531, 'GCC': 12638, 'GCG': 12752, 'GCT': 14778, 'GGA': 14344, 'GGC': 12652, 'GGG': 12372, 'GGT': 14880, 'GTA': 19456, 'GTC': 12630, 'GTG': 15206, 'GTT': 14905, 'TAA': 22747, 'TAC': 9985, 'TAG': 21251, 'TAT': 20200, 'TCA': 

In [15]:
nc_counts = make_dict_upto_K(MAX_K)
for sample in nc_all:
    update_count_one_K(nc_counts,MAX_K,sample,True)
harvest_counts_from_K(nc_counts,MAX_K)
print("NC counts:\n",nc_counts)
nc_freqs = count_to_frequency(nc_counts,MAX_K)
print ("Frequency:\n",nc_freqs)

NC counts:
 {'A': 282620, 'C': 208784, 'G': 261021, 'T': 271575, 'AA': 69759, 'AC': 52031, 'AG': 62181, 'AT': 90276, 'CA': 54041, 'CC': 46460, 'CG': 46531, 'CT': 54003, 'GA': 74290, 'GC': 56535, 'GG': 56523, 'GT': 65439, 'TA': 74825, 'TC': 46381, 'TG': 88313, 'TT': 54412, 'AAA': 15953, 'AAC': 14290, 'AAG': 14199, 'AAT': 22905, 'ACA': 13428, 'ACC': 11615, 'ACG': 11469, 'ACT': 13432, 'AGA': 15759, 'AGC': 14100, 'AGG': 14121, 'AGT': 15745, 'ATA': 18068, 'ATC': 11631, 'ATG': 45194, 'ATT': 13320, 'CAA': 12097, 'CAC': 10478, 'CAG': 10247, 'CAT': 19458, 'CCA': 11971, 'CCC': 10193, 'CCG': 10427, 'CCT': 11998, 'CGA': 12114, 'CGC': 10355, 'CGG': 10240, 'CGT': 11922, 'CTA': 16947, 'CTC': 10238, 'CTG': 12727, 'CTT': 12208, 'GAA': 17139, 'GAC': 15085, 'GAG': 15142, 'GAT': 24643, 'GCA': 14480, 'GCC': 12588, 'GCG': 12700, 'GCT': 14831, 'GGA': 14352, 'GGC': 12686, 'GGG': 12689, 'GGT': 14828, 'GTA': 20533, 'GTC': 12466, 'GTG': 15755, 'GTT': 14768, 'TAA': 22409, 'TAC': 10425, 'TAG': 20851, 'TAT': 19221,